








# Notebooks as Tidy ~~Developer~~ Data Experience
The Python object protocol, extended by the jedi and IPython completer features, provide a rich API for interactive computing, with standout perfomances by `pandas.DataFrame`s.

Turning this capability on its head, we can use tidy data sources to extend our environment. Let's use some `nbformat` JSON as our dataset, and expose it in the completer.

![](https://user-images.githubusercontent.com/4236275/43682085-7db479dc-9836-11e8-90f4-df1ce542bbb0.PNG)

In [1]:
    import deathbeds
    import pandas as pd
    from pathlib import Path

Since the notebook is the json format we can use `pandas.read_json` method.

In [2]:
    df = pd.concat({
        path: pd.read_json(path, typ='Series')
        for path in Path(deathbeds.__file__).parent.glob(
            '[0-9]*-*.ipynb')}).unstack(-1)

In this example, we trust that the notebooks are valid `nbformat.v4` schemas.

In [3]:
    def test_nbformat():
        from nbformat import read
        g = df.index.to_series().apply(str).apply(
            read, as_version=4).apply(pd.Series)

    if __name__ == '__main__':
        !ipython -m pytest -- -p no:pytest-importnb Untitled11.ipynb

In [4]:
    cells = (df.cells.apply(pd.Series).stack()
             .apply(pd.Series).set_index('cell_type', append=True))

In [5]:
    source, output =  (cells[object].apply(pd.Series).stack() for object in ('source', 'outputs'))

In [6]:
     def predict(str): return source[
         source.str.lstrip().str.startswith(str.strip())].str.strip().drop_duplicates().tolist()

In [7]:
     def event(self, event): return predict(event.line)

In [8]:
    def load_ipython_extension(ip):
        ip.set_hook('complete_command', event, re_key=".*")
        
    ø = __name__ == '__main__'
    if ø: load_ipython_extension(get_ipython())

For some reason we haven't used our style in any posts.  By adding them to this post we can now using a little bit of the IPython completion to predict our prior code. Or MarkDown. Or tests.

In [9]:
    import __style__
    __import__('__style__')

<module '__style__' from 'C:\\Users\\deathbeds\\deathbeds.github.io\\deathbeds\\__style__.ipynb'>

* Another invocation of this idea would be a widget.

In [10]:
    if ø:
        import ipywidgets, traitlets, IPython
        finder = ipywidgets.VBox([ipywidgets.Text(), ipywidgets.SelectMultiple()])
        traitlets.dlink((finder.children[0], "value"), (finder.children[1], "options"), 
                        lambda x: [(object, object) for object in predict(x)])

        IPython.display.display(finder)